<a href="https://colab.research.google.com/github/Fukkatsuso/livedoornews-topicmodel/blob/master/topicmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# トピックモデル

## Goal
- ライブドアコーパスでWeb記事分類器を作る

## Step
1. ライブドアコーパスをスクレイピング
1. データの前処理
  1. 形態素解析 => **MeCab** (+NEologd)
  1. 不要語の削除, 語の統一(ステミング)
1. トピックモデルの構築 => **gensim**
1. 機械学習 => **sklearn**

## 参考
- [LDAによるトピックモデル with gensim ~ Qiitaのタグからユーザーの嗜好を考える ~](https://qiita.com/shizuma/items/44c016812552ba8a8b88)
- [トピックモデルをザックリと理解してサクッと試した](https://qiita.com/d-ogawa/items/c423cd4b01c6ed84a5e7)
- [WordCloudとpyLDAvisによるLDAの可視化について](http://www.ie110704.net/2018/12/29/wordcloud%E3%81%A8pyldavis%E3%81%AB%E3%82%88%E3%82%8Blda%E3%81%AE%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6/)
- [自然言語処理による文書分類の基礎の基礎、トピックモデルを学ぶ](https://qiita.com/icoxfog417/items/7c944cb29dd7cdf5e2b1)
- [scikit-learnとgensimでニュース記事を分類する](https://qiita.com/yasunori/items/31a23eb259482e4824e2)
- [文書分類で自然言語処理に触れる](https://colab.research.google.com/drive/1IMjc-RTesapfNCEh0TPmg_ce_qAcV95b#scrollTo=9a9CUjgUXgB6)
- [自然言語処理における前処理の種類とその威力](https://qiita.com/Hironsan/items/2466fe0f344115aff177)
- [Python3×日本語：自然言語処理の前処理まとめ](https://qiita.com/chamao/items/7edaba62b120a660657e)
- [ニュース記事の分類を機械学習で予測する](https://qiita.com/hyo_07/items/ba3d53868b2f55ed9941)


## データ収集
### 対象
- [livedoorニュースコーパス](https://www.rondhuit.com/download.html#ldcc)
  - [トピックニュース](http://news.livedoor.com/category/vender/news/)
  - [Sports Watch](http://news.livedoor.com/category/vender/208/)
  - [ITライフハック](http://news.livedoor.com/category/vender/223/)
  - [家電チャンネル](http://news.livedoor.com/category/vender/kadench/)
  - [MOVIE ENTER](http://news.livedoor.com/category/vender/movie_enter/)
  - [独女通信](http://news.livedoor.com/category/vender/90/)
  - [エスマックス](http://news.livedoor.com/category/vender/smax/)
  - [livedoor HOMME](http://news.livedoor.com/category/vender/homme/)
  - [Peachy](http://news.livedoor.com/category/vender/ldgirls/)


In [0]:
# Get dataset
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!mkdir -p dataset/livedoor && tar xvzf ldcc-20140209.tar.gz -C /content/dataset/livedoor --strip-components 1
!rm ldcc-20140209.tar.gz

# Install MeCab
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

# Install mecab-ipadic-NEologd
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

!pip install mojimoji

In [0]:
# 確認
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

## 1.前処理なしLDA

In [0]:
import gensim
import glob2
import MeCab

mecab = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

paths = glob2.glob("dataset/livedoor/sports-watch/*-*.txt")

In [0]:
words = []
for path in paths:
  data = open(path, 'r', encoding="utf-8").read().split('\n')
  title = data[2]
  words.append(mecab.parse(title).split())

In [0]:
# 辞書, コーパス作成
dictionary = gensim.corpora.Dictionary(words)

dictionary.save_as_text("dictionary1.dict.txt")

corpus = [dictionary.doc2bow(w) for w in words]

In [0]:
# LDA
topic_N = 10
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=topic_N, id2word=dictionary)

for i in range(topic_N):
  print('TOPIC:', i, '=>', lda.print_topic(i))

using symmetric alpha at 0.1
using symmetric eta at 0.1
using serial LDA version on this node
running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 900 documents, updating model once every 900 documents, evaluating perplexity every 900 documents, iterating 50x with a convergence threshold of 0.001000
too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
-11.454 per-word bound, 2805.7 perplexity estimate based on a held-out corpus of 900 documents with 14123 words
PROGRESS: pass 0, at document #900/900
topic #0 (0.100): 0.053*"の" + 0.039*"】" + 0.039*"に" + 0.039*"【" + 0.038*"Watch" + 0.037*"Sports" + 0.033*"「" + 0.032*"」" + 0.030*"、" + 0.023*"が"
topic #4 (0.100): 0.036*"「" + 0.036*"」" + 0.032*"に" + 0.031*"が" + 0.024*"は" + 0.021*"の" + 0.017*"・" + 0.016*"Sports" + 0.016*"【" + 0.016*"Watch"
topic #2 (0.100): 0.036*"、" + 0.030*"Sports" + 0.030*"【" + 0.030*"Watch" + 0.029*"】" + 0.027*"「" + 

TOPIC: 0 => 0.053*"の" + 0.039*"】" + 0.039*"に" + 0.039*"【" + 0.038*"Watch" + 0.037*"Sports" + 0.033*"「" + 0.032*"」" + 0.030*"、" + 0.023*"が"
TOPIC: 1 => 0.022*"に" + 0.009*"Sports" + 0.009*"Watch" + 0.009*"【" + 0.009*"た" + 0.009*"、" + 0.008*"】" + 0.007*"”" + 0.007*"“" + 0.007*"ない"
TOPIC: 2 => 0.036*"、" + 0.030*"Sports" + 0.030*"【" + 0.030*"Watch" + 0.029*"】" + 0.027*"「" + 0.026*"」" + 0.022*"に" + 0.019*"を" + 0.014*"た"
TOPIC: 3 => 0.035*"の" + 0.027*"」" + 0.026*"「" + 0.025*"は" + 0.025*"Sports" + 0.024*"【" + 0.024*"Watch" + 0.024*"】" + 0.021*"、" + 0.021*"た"
TOPIC: 4 => 0.036*"「" + 0.036*"」" + 0.032*"に" + 0.031*"が" + 0.024*"は" + 0.021*"の" + 0.017*"・" + 0.016*"Sports" + 0.016*"【" + 0.016*"Watch"
TOPIC: 5 => 0.040*"】" + 0.040*"、" + 0.039*"Sports" + 0.039*"Watch" + 0.039*"【" + 0.038*"の" + 0.023*"は" + 0.022*"に" + 0.020*"「" + 0.020*"」"
TOPIC: 6 => 0.030*"・" + 0.028*"の" + 0.028*"は" + 0.021*"Sports" + 0.021*"、" + 0.021*"【" + 0.020*"】" + 0.020*"Watch" + 0.020*"に" + 0.012*"と"
TOPIC: 7 => 0.047*"、" + 0.

## 2.前処理ありLDA

1. 正規化
  - 半角かな => 全角かな
  - 全角英数 => 半角英数
  - 大文字 => 小文字
  - 辞書による統一?
1. 品詞で取捨選択
1. ストップワード除去
  - 辞書
    - [SlothLib](http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt)

In [0]:
import gensim
import glob2
import mojimoji
import MeCab
import urllib3

from sklearn.model_selection import train_test_split

mecab = MeCab.Tagger("mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

paths = glob2.glob("dataset/livedoor/*/*-*.txt")

# 学習用と評価用に分ける
train_rate = 0.8
random_state = 5903
train_article_paths, test_article_paths = train_test_split(paths, train_size=train_rate, random_state=random_state)

In [0]:
# カテゴリごとの記事数の分散
def deviation(random_state, test_article_paths):
  cat = {}
  for path in test_article_paths:
    c = path.split('/')[2]
    if cat.get(c) == None:
      cat[c] = 0
    else:
      cat[c] += 1

  avr = 0
  for c in cat:
    avr += cat[c]
  avr /= len(cat)

  d = 0
  for c in cat:
    d += (cat[c]-avr)**2

  return d

In [0]:
# パースされた文字列から品詞を限定して取り出す
def extract_by_parts(parsed, parts):
  words = []
  lines = parsed.split('\n')
  for line in lines:
    feature = line.split('\t')
    if len(feature) == 2:
      info = feature[1].split(',')
      if info[0] in parts:
        if info[6] == '*': 
          words.append(feature[0])  # 活用なしの語
        else:  
          words.append(info[6]) # 表記ゆれの対処
  return words

# 各単語を前処理にかける
def preprocess_words(words, stopwords):
  for i in range(len(words)):
    words[i] = unify_chartype(words[i])
  words = filter_stopwords(words, stopwords)
  return words

# 文字種を統一する
def unify_chartype(text):
  text = mojimoji.zen_to_han(text, kana=False, digit=True, ascii=True) # 全角英数=>半角英数
  text = mojimoji.han_to_zen(text, kana=True, digit=False, ascii=False) # 半角かな=>全角かな
  text = text.lower() # 大文字=>小文字
  return text

# ストップワードを除去する
def filter_stopwords(words, stopwords):
  filtered_words = [word for word in words if word not in stopwords]
  return filtered_words

# ストップワードのリストを返す
def get_stopwords():
  # SlothLib
  slothlib_url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
  http = urllib3.PoolManager()
  res = http.request('GET', slothlib_url)
  stopwords = res.data.decode('utf-8').split()
  return stopwords

In [0]:
def text2words(text, stopwords, parts):
  parsed_text = mecab.parse(text)
  words = extract_by_parts(parsed_text, parts)
  words = preprocess_words(words, stopwords)
  return words

stopwords = get_stopwords()

train_words = []
for path in train_article_paths:
  data = open(path, 'r', encoding="utf-8").read().split('\n')
  words = text2words(data[2], stopwords, ('名詞'))
  train_words.append(words)

In [0]:
def words2corpus(dictionary, words):
  return [dictionary.doc2bow(w) for w in words]

# 辞書, コーパス作成
dictionary = gensim.corpora.Dictionary(train_words)
dictionary.filter_extremes(no_below=5, no_above=0.075)
dictionary.save_as_text("dictionary2.dict.txt")

train_corpus = words2corpus(dictionary, train_words)

In [74]:
# LDA
topic_N = 10
lda = gensim.models.ldamodel.LdaModel(corpus=train_corpus, num_topics=topic_N, id2word=dictionary)

for i in range(topic_N):
  print('TOPIC:', i, '=>', lda.print_topic(i))

TOPIC: 0 => 0.019*"映画" + 0.018*"女子" + 0.017*"google" + 0.016*"週間ランキング" + 0.015*"理由" + 0.014*"ビデオsalon" + 0.013*"恋愛" + 0.013*"ロゴ" + 0.011*"年収" + 0.011*"公開"
TOPIC: 1 => 0.021*"登場" + 0.019*"人気" + 0.018*"watch" + 0.018*"sports" + 0.014*"チェック" + 0.013*"ネット" + 0.011*"発売" + 0.011*"売れ筋" + 0.010*"声" + 0.009*"初"
TOPIC: 2 => 0.029*"プレゼント" + 0.023*"iphone" + 0.023*"終了" + 0.014*"キャンペーン" + 0.013*"映画" + 0.011*"撮影" + 0.010*"世界" + 0.010*"チャンス" + 0.010*"虎の巻" + 0.009*"カメラ"
TOPIC: 3 => 0.040*"watch" + 0.040*"sports" + 0.039*"独女" + 0.022*"氏" + 0.020*"批判" + 0.016*"殺到" + 0.016*"映画" + 0.015*"韓国" + 0.015*"オトナ女子" + 0.009*"監督"
TOPIC: 4 => 0.043*"watch" + 0.043*"sports" + 0.020*"更新" + 0.020*"提供開始" + 0.020*"ソフトウェア" + 0.018*"smartphone" + 0.016*"au" + 0.015*"写真" + 0.014*"不具合" + 0.014*"向け"
TOPIC: 5 => 0.028*"チェック" + 0.020*"搭載" + 0.019*"売れ筋" + 0.019*"発売" + 0.017*"レポート" + 0.017*"タブレット" + 0.016*"発表" + 0.014*"0" + 0.014*"1" + 0.013*"nttドコモ"
TOPIC: 6 => 0.029*"deji" + 0.019*"発表" + 0.018*"androidアプリ" + 0.016*"対応" + 0.014*

## 分類

In [75]:
# テスト用データ
test_words = []
for path in test_article_paths:
  data = open(path, 'r', encoding="utf-8").read().split('\n')
  words = text2words(data[2], stopwords, ('名詞'))
  test_words.append(words)

test_corpus = words2corpus(dictionary, test_words)

for i in range(len(test_article_paths)):
  category = test_article_paths[i].split('/')[2]
  title = open(test_article_paths[i], 'r', encoding="utf-8").read().split('\n')[2]
  print(category, '\n', title, '\n', lda[test_corpus[i]], '\n')

ストリーミング出力は最後の 5000 行に切り捨てられました。
 ノムさんが、松井秀喜の日本球界を願う理由 
 [(0, 0.23996781), (1, 0.49335313), (2, 0.033335026), (3, 0.03333433), (4, 0.03333415), (5, 0.033333335), (6, 0.033337627), (7, 0.0333377), (8, 0.03333341), (9, 0.033333454)] 

sports-watch 
 川澄奈穂美が絶賛する美女選手＝「女ジダン」とは？ 
 [(0, 0.025000563), (1, 0.025002263), (2, 0.025000768), (3, 0.025000533), (4, 0.025003456), (5, 0.025010824), (6, 0.025000092), (7, 0.29083675), (8, 0.025000136), (9, 0.5091446)] 

it-life-hack 
 ゲーム機に必要なストレージ容量は？ ダウンロード販売が本格化【デジ通】 
 [(0, 0.3235036), (1, 0.025002116), (2, 0.025000464), (3, 0.02500049), (4, 0.025005668), (5, 0.025000764), (6, 0.21017371), (7, 0.02500789), (8, 0.29130316), (9, 0.02500218)] 

dokujo-tsushin 
 キスシーンだらけのDVD、あなたはウットリしますか？ 
 [(0, 0.36664066), (1, 0.033336785), (2, 0.033333335), (3, 0.033333335), (4, 0.03333678), (5, 0.033333335), (6, 0.033333335), (7, 0.033341963), (8, 0.033343814), (9, 0.36666667)] 

it-life-hack 
 ソーシャルの今が分かる！ヤフーの「話題なう」ってなに？ 
 [(0, 0.050000004), (1, 0.050000004), (2, 0.050